In [1]:
import os
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import scipy


def inite(elec,orb):
    config=[]
    list1=[]
    #singles
    for x in range(elec):
        count=orb-elec
        while (count<orb):
            for e in range(elec):
                if x==e:
                    if x%2==0:
                        config.append(count)
                        count=count+2
                    else:
                        config.append(count+1)
                        count=count+2
                else:
                    config.append(e)
            list1.append(config)
            config=[]
    #doubles
    for x in range(elec):
        for y in range(x+1,elec):
            count1=orb-elec
            count2=orb-elec
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont=0
                    if count1==count2:
                        if (x%2)!=(y%2):
                            cont=1
                    else:
                        cont=1
                    if (x%2)==(y%2) and count2<count1:
                        cont=0
                    if cont==1:    
                        for e in range(elec):
                            if x==e:
                                if x%2==0:
                                    config.append(count1)
                                else:
                                    config.append(count1+1)
                            elif y==e:
                                if y%2==0:
                                    config.append(count2)
                                else:
                                    config.append(count2+1)
                            else:
                                config.append(e)
                        list1.append(config)
                        config=[]
    return list1

def gs_exact(symbols, geometry, electrons, charge, shots=None, max_iter=100):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, charge=charge)
    hf_state = qml.qchem.hf_state(electrons, qubits)
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

    wires=range(qubits)
    params = np.zeros(len(singles) + len(doubles))
    # Define the device
    dev = qml.device("default.qubit", wires=qubits, shots=shots)
    
    # Define the qnode
    if shots==None:
        @qml.qnode(dev, interface="autograd", diff_method="adjoint")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)
    else:
        @qml.qnode(dev, interface="autograd")
        def circuit(params, wires, s_wires, d_wires, hf_state):
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)

    optimizer = qml.GradientDescentOptimizer(stepsize=2.0)
    for n in range(max_iter):
        params, energy = optimizer.step_and_cost(circuit, params, wires=range(qubits),
                                                 s_wires=s_wires, d_wires=d_wires,
                                                 hf_state=hf_state)
    
    print(circuit(params, wires, s_wires, d_wires, hf_state))
    return params
def ee_exact(symbols, geometry, electrons, charge,params,shots=0):
    # Build the electronic Hamiltonian
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry, charge=charge)
    hf_state = qml.qchem.hf_state(electrons, qubits)
    singles, doubles = qml.qchem.excitations(electrons, qubits)
    # Map excitations to the wires the UCCSD circuit will act on
    s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
    wires=range(qubits)

    null_state = np.zeros(qubits,int)
    list1 = inite(electrons,qubits)
    values =[]
    for t in range(1):
        if shots==0:
            #dev = qml.device("default.qubit", wires=qubits)
            dev = qml.device("default.qubit", wires=qubits)
        else:
            #dev = qml.device("default.qubit", wires=qubits,shots=shots)
            dev = qml.device("default.qubit", wires=qubits,shots=shots)
        #circuit for diagonal part
        @qml.qnode(dev)
        def circuit_d(params, occ,wires, s_wires, d_wires, hf_state):
            for w in occ:
                qml.X(wires=w)
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)
        #circuit for off-diagonal part
        @qml.qnode(dev)
        def circuit_od(params, occ1, occ2,wires, s_wires, d_wires, hf_state):
            for w in occ1:
                qml.X(wires=w)
            first=-1
            for v in occ2:
                if v not in occ1:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for v in occ1:
                if v not in occ2:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
            return qml.expval(H)
        #final M matrix
        M = np.zeros((len(list1),len(list1)))
        for i in range(len(list1)):
            for j in range(len(list1)):
                if i == j:
                    M[i,i] = circuit_d(params, list1[i], wires, s_wires, d_wires, null_state)
        print("diagonal parts done")
        for i in range(len(list1)):
            for j in range(len(list1)):
                if i!=j:
                    Mtmp = circuit_od(params, list1[i],list1[j],wires, s_wires, d_wires, null_state)
                    M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
        print("off diagonal terms done")
        #ERROR:not subtracting the gs energy
        eig,evec=np.linalg.eig(M)
        values.append(np.sort(eig))
    return values


r = 1.88973  # 1 Angstrom in Bohr
# Define the molecule
symbols  = ['H', 'H', 'H', 'H']
geometry = np.array([[0.0, 0.0, 0.0],
                     [0.0, 0.0, 1.5*r],
                     [0.0, 1.5*r, 0.0],
                     [0.0, 1.5*r, 1.5*r]], requires_grad = False)
electrons = 4
orbitals = 8  # Number of orbitals

#in bohr 
#symbols  = [ 'H', 'H']
#geometry = np.array([[0.0, 0.0,  0.0],
#                     [0.0, 0.0,  2.83459]], requires_grad = False)
#electrons = 2
charge = 0
params = gs_exact(symbols,geometry,electrons,charge,max_iter=500)
#params_exact=params

eig = ee_exact(symbols, geometry, electrons, charge,params)
print('exact eigenvalues:\n', eig)

-1.951956244782695


/Users/prince.kwao/miniconda3/envs/cwp/lib/python3.13/site-packages/pennylane/ops/op_math/composite.py:211: FutureWarning: functools.partial will be a method descriptor in future Python versions; wrap it in staticmethod() if you want to preserve the old behavior
  return self._math_op(math.vstack(eigvals), axis=0)


diagonal parts done
off diagonal terms done
exact eigenvalues:
 [tensor([-1.89853267, -1.80374152, -1.79717922, -1.77023601, -1.70223977,
        -1.62842371, -1.43179084, -1.42416976, -1.41790215, -1.41162145,
        -1.35269283, -1.34752374, -1.30037573, -1.2861468 , -1.2333312 ,
        -1.21969272, -1.21849968, -1.12153776, -1.11834082, -1.10556786,
        -1.02453828, -0.91221895, -0.90671382, -0.67388813, -0.46464919,
        -0.44029814], requires_grad=True)]


In [2]:
eig1 = ee_exact(symbols, geometry, electrons, charge,params,shots=1000)
print('eigenvalue 1kshots:\n', np.array(eig1))

diagonal parts done
off diagonal terms done
eigenvalue 1kshots:
 [[-1.9211958 +0.j         -1.81991265+0.j         -1.78781634+0.j
  -1.75237686+0.j         -1.67131071+0.j         -1.62129574+0.j
  -1.43678815+0.j         -1.42637377-0.03251729j -1.42637377+0.03251729j
  -1.42317063+0.j         -1.3637343 +0.j         -1.33579857+0.j
  -1.29940516-0.0132026j  -1.29940516+0.0132026j  -1.24604648+0.j
  -1.20423449+0.j         -1.19086105+0.j         -1.14499609+0.j
  -1.1229736 +0.j         -1.09412815+0.j         -1.03228718+0.j
  -0.92070177+0.j         -0.88715486+0.j         -0.70946227+0.j
  -0.46801633+0.j         -0.43093988+0.j        ]]
